# Variable DC-link voltages

Since there are not only ideal sources in the regular grid, alternative, variable DC-link voltages shall be presented here. The models to be used will be discussed in detail and it will be explained how they can be created and used in the Env.

- ### Modelling a PV-Array,
- ### Modelling a Batterie,
- ### Access the models.

In [1]:
using ElectricGrid
using PlotlyJS

[ Info: Precompiling ElectricGrid [d6749236-802c-4af3-9b09-2fcebf7269d7]


WebIO._IJuliaInit()

## Modelling a PV-Array

This notebook will explain the PV arrays structurally and how they were implemented.

As a basis serves the [paper](https://doi.org/10.1016/j.nrjag.2014.04.001) which explains the physical characteristics of a PV module in more detail. In order to avoid further calculations at runtime, a simpler approximation is used, the ideal single diode model. This neglects the parallel and series resistance of the PV module.

In the following we define a mutable struct with the most important data of a PV module, most of the data are given by the data sheets.

In [2]:
ModulePV = SolarModule(); # For this example we use the default values

LoadError: UndefVarError: SolarModule not defined

As a rule, many of the modules are initially connected in series, especially in large systems, in order to increase the voltage. Further modules can then be added in parallel to further increase the current.

We therefore define another mutable struc, which inherits the properties of the PV module and specifies how many modules are to be created in series and then again in parallel to it.

For real applications, this often leads to problems, since shadowing of individual cells (!!!) can result in a large drop in power. Therefore, in practice, countermeasures are often taken, such as the integration of bypass diodes.

In [3]:
ArrayPV = SolarArray(;ModuleParameters=ModulePV, serial=1, parallel=1)

LoadError: UndefVarError: ModulePV not defined

Now let us take a look  characteristics for different irradiations. Therefore, we define a function that gives us the current as a function of voltage, irradiation and temperature. In addition, a function should also output the voltage as a function of current, irradiation and temperature, since this will later be interesting for our application in ElectricGrid.jl.

### Function for module

For the purpose of this notebook, the functions here are defined vectorially so that they can be evaluated directly for multiple irradiances and voltages.

The functions that are later used in the `ElectricGrid.jl` environment return only a scalar value.

In [4]:
function GetI_vec(SolarArr::SolarArray, V, G, T)
    self = SolarArr.ModuleParameters
    
    function I_photo(self::SolarModule, G, T)
        dT = self.T_0 + T
        I_ph = G./self.G_ref*(self.I_ph_ref + self.mu_sc * dT).* ones(length(V))'
        return I_ph
    end;

    function I_diode(self::SolarModule, V, G, T)
        dT = self.T_0 + T
        V_T = self.k*dT/self.q
        I_d = self.I_0 * (exp.(V  / (self.ni * self.N_cell * SolarArr.serial * V_T)).-1).*ones(5)'
        return I_d
    end;
    
    I = (I_photo(self, G, T)' - I_diode(self, V, G, T)) * SolarArr.parallel
    return I
end


LoadError: UndefVarError: SolarModule not defined

### Characteristics for PV modules

Here the values are defined for which the modules are to be evaluated.

In [5]:
T = 25
G = collect(200:200:1000)
V = collect(0:0.1:30);

In [6]:
i = GetI_vec(ArrayPV, V, G, T);

LoadError: UndefVarError: ArrayPV not defined

In [7]:
labels = ["200 W/m^2" "400 W/m^2" "600 W/m^2" "800 W/m^2" "1000 W/m^2"];
plot([scatter(x=V, y=i[:,1], mode="lines", name=labels[1]),
scatter(x=V, y=i[:,2], mode="lines", name=labels[2]),
scatter(x=V, y=i[:,3], mode="lines", name=labels[3]),
scatter(x=V, y=i[:,4], mode="lines", name=labels[4]),
scatter(x=V, y=i[:,5], mode="lines", name=labels[5])],
Layout(xaxis_range=[0,30], yaxis_range=[0,5], title="PV module - I(V)", xaxis_title="V", yaxis_title="I"))

LoadError: UndefVarError: i not defined

In [8]:
V_ = V.*ones(5)';
P = i .* V_;

LoadError: UndefVarError: i not defined

In [9]:
plot([
scatter(x=V, y=P[:,1], mode="lines", name=labels[1]),
scatter(x=V, y=P[:,2], mode="lines", name=labels[2]),
scatter(x=V, y=P[:,3], mode="lines", name=labels[3]),
scatter(x=V, y=P[:,4], mode="lines", name=labels[4]),
scatter(x=V, y=P[:,5], mode="lines", name=labels[5])],
Layout(xaxis_range=[0,30], yaxis_range=[0,100], title="PV module - P(V)", xaxis_title="V", yaxis_title="P"))

LoadError: UndefVarError: P not defined

For the V(I) characteristic, the axes are swapped during plotting and the explicit function is not used. When plotting with the explicit function, there is a problem with the logarithm, which must not be less than zero. Therefore, the range of values must be very fine tuned. But since we can see in the upper two plots how steep the function is, this is very difficult.

In [10]:
plot([
scatter(x=i[:,1], y=V, mode="lines", name=labels[1]),
scatter(x=i[:,2], y=V, mode="lines", name=labels[2]),
scatter(x=i[:,3], y=V, mode="lines", name=labels[3]),
scatter(x=i[:,4], y=V, mode="lines", name=labels[4]),
scatter(x=i[:,5], y=V, mode="lines", name=labels[5])],
Layout(xaxis_range=[0,5], yaxis_range=[0,30], title="PV module - V(I)", xaxis_title="I", yaxis_title="V"))

LoadError: UndefVarError: i not defined

In [11]:
plot([
scatter(x=i[:,1], y=P[:,1], mode="lines", name=labels[1]),
scatter(x=i[:,2], y=P[:,2], mode="lines", name=labels[2]),
scatter(x=i[:,3], y=P[:,3], mode="lines", name=labels[3]),
scatter(x=i[:,4], y=P[:,4], mode="lines", name=labels[4]),
scatter(x=i[:,5], y=P[:,5], mode="lines", name=labels[5])],
Layout(xaxis_range=[0,5], yaxis_range=[0,80], title="PV module - P(I)", xaxis_title="I", yaxis_title="P"))

LoadError: UndefVarError: i not defined

### Select Test point

Here we use the functions that are also part of ElectricGrid.jl to evaluate the characteristic curves at selected points.

In [12]:
v_test = 22
i_test = get_I(ArrayPV, v_test, 1000, 27)

LoadError: UndefVarError: ArrayPV not defined

In [13]:
i_test2 = 3.42674573464278
v_test2 = get_V(ArrayPV, i_test2, 1000, 27)

LoadError: UndefVarError: ArrayPV not defined

In [14]:
Plot([scatter(x=V, y=i[:,5], mode="lines", name=labels[1]),
      scatter(x=[v_test], y=[i_test], mode="markers", name="Test point",marker=attr(size=10))], 
Layout(xaxis_range=[0,30], yaxis_range=[0,5], title="PV module - I(V)", xaxis_title="V", yaxis_title="I"))

LoadError: UndefVarError: i not defined

In [15]:
Plot([scatter(x=i[:,5], y=V, mode="lines", name=labels[1]),
      scatter(x=[i_test2], y=[v_test2], mode="markers", name="Test point",marker=attr(size=10))],
Layout(xaxis_range=[0,5], yaxis_range=[0,30], title="PV module - V(I)", xaxis_title="I", yaxis_title="V"))

LoadError: UndefVarError: i not defined

## Modelling batteries

The modeling of the battery follows that described in the implementation in [this papper](https://de.mathworks.com/content/dam/mathworks/tag-team/Objects/i/71900-ieee-2012-high-fidelity-lithium-battery-model-with-thermal-effect.pdf). The battery is a lithium-ion battery, which is shown in the following equivalent circuit.

Figure

The most important components are the input resistance $R_0$ and the RC pairs, which reflect the individual cells within the battery. The number of the cell is indicated by the index $n$. all quantities of the battery are dependent on the state of charge (SOC) and the temperature of the battery. Since these are nonlinear characteristics, they must be recorded experimentally and stored in look-up tables. Here, the values from the Matlab implementation were also used ([Source](https://de.mathworks.com/help/autoblks/ref/equivalentcircuitbattery.html)).

Now we create a battery. To do this, we first store the characteristic curves for the individual cell. If no specific values are entered here, the default values described above are used.

In [16]:
Cell = battery_module(); #Blabla

In [17]:
Battery = battery_block(battery_module=Cell);

Now that the battery has been created, a charge and final charge cycle is to be plotted in the following. By default, the SOC is set to 0. The battery can be accessed via the update function. With this function, the charge and discharge current can be defined and the temperature of the cell can be adjusted. The update function is called in every step.

In [18]:
I_s = 13; # Define a current of 50 mA for charging and discharging

SOC_load = []
Voltage_load = []
Current_load = []

SOC_discharge = []
Voltage_discharge = []
Current_discharge = []

Any[]

In [19]:
for i in 1:1e7*8 # Ts*Steps=T_total // 1e-4 * 1e7 * 7,4 =  
    I_in = -I_s

    update_bat(Battery, I_in, 20)

    append!(SOC_load, Battery.SOC)
    append!(Current_load, I_in)
    append!(Voltage_load, Battery.V)
end

for i in 1:1e7*8
    I_in = I_s

    update_bat(Battery, I_in, 20)

    append!(SOC_discharge, Battery.SOC)
    append!(Current_discharge, I_in)
    append!(Voltage_discharge, Battery.V)
end

LoadError: UndefVarError: update_bat not defined

In [20]:
plot([scatter(x=SOC_load[1:100000:end], y=Voltage_load[1:100000:end], mode="markers", name="Charge"),
scatter(x=SOC_discharge[1:100000:end], y=Voltage_discharge[1:100000:end], mode="markers", name="Discharge")],
Layout(title="Charge and discharge characteristic", xaxis_title="SOC", yaxis_title="V"))

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

## Access the models

As already shown in the basic enviroment notebooks, we now want to create an enviroment that uses the presented models instead of the ideal sources. The first step is to create an `env` with the corresponding parameter dict entries, specifying the DC link. In the first example we want to have a fully charged battery being discharged over a load.

In [21]:
CM = [ 0. 1.
      -1. 0.]

R_load, L_load, X, Z = ParallelLoadImpedance(1e3, .95, 230)


parameters = Dict{Any,Any}(
    "source" => Any[
        Dict{Any,Any}("source_type" => "battery", "pwr" => 15000.0, "fltr" => "L",
                      "L1" => 0.001, "C" => 2e-8, "L2" => 0.001, "R_C" => 0.01,
                      "R2" => 0.05, "Q" => 100 * 26 * 3600, "mode" => 1,
                      "parallel" => 48, "series" => 160, "i_bat_limit" => 1e3,
                      "i_limit" => 1e3, "v_limit" => 1e3),
            ],
    "load" => Any[
        Dict{Any,Any}("impedance" => "RL", "R" => R_load, "L" => L_load, "i_limit" => 500,
                      "v_limit" => 1000)
    ],
    "cable" => Any[
        Dict{Any,Any}("Cb" => 4.0e-9, "Lb" => 0.000264, "Rb" => 0.002, "C" => 0.4e-2, "i_limit" => 1.0e13, "v_limit" => 1000, "len" => 1.0, "L" => 0.25e-5, "R" => 0.1e-1),
    ],
    "grid" => Dict{Any,Any}("fs" => 1e4, "phase" => 3, "v_rms" => 230, "f_grid" => 50, "ramp_end" => 0.9)
)


states = []
states_all = []
Charging = []
Discharging = []
V_dc_char = []
V_dc_dis = []
env = ElectricGridEnv(num_sources=1, num_loads=1, CM=CM, parameters=parameters, t_end=2, verbosity=2)

Multi_Agent = SetupAgents(env)

hook = DataHook(collect_vdc_ids = [1],
                collect_soc_ids = [1],
                collect_idc_ids = [1],)

hook = Simulate(Multi_Agent, env, hook=hook)

LoadError: KeyError: key "serial" not found

Below we plot the results:

In [22]:
p = RenderHookResults(hook = hook,
                    vdc_to_plot     = [1],
                    soc_to_plot     = [1],
                    idc_to_plot     = [1],
                    return_plot     = false)

LoadError: UndefVarError: hook not defined

The struct `vdc_link_voltages` collects all information of the sources in a list:

In [23]:
env.vdc_link_voltages.sources

LoadError: UndefVarError: env not defined

In the second example we want to create another PV array, which should supply a load.

In [24]:
CM = [ 0.  1.
      -1.  0.]

R_load, L_load, X, Z = ParallelLoadImpedance(1e3, .95, 230)

parameters = Dict{Any, Any}(
        "source" => Any[
                        Dict{Any, Any}("source_type" => "pv", "fltr"=>"L",
                                       "L1"=>0.001, "C"=>2e-8, "L2"=>0.001, "R_C"=>0.01,
                                       "R2"=> 0.05, "mode" => 1, "parallel" => 256, "serial" => 512,),
                        ],
        "load"   => Any[
                        Dict{Any, Any}("impedance" => "RL", "R" => R_load, "L" => L_load,
                                       "i_limit" => 500, "v_limit" => 1000),
                        ],
        "cable"   => Any[
                        Dict{Any, Any}("R" => 1e-3, "L" => 1e-4, "C" => 1e-4,
                                       "i_limit" => 1e4, "v_limit" => 1e4,),
                        ],
        "grid" => Dict{Any, Any}("fs"=>1e4, "phase"=>3, "v_rms"=>230, "f_grid" => 50,
                                 "ramp_end" => 0.9)
    )


env = ElectricGridEnv(num_sources=1, num_loads=1, CM = CM, parameters = parameters, t_end=2, verbosity=2)

Multi_Agent = SetupAgents(env)

hook = DataHook(collect_vdc_ids=[1],
                collect_soc_ids=[1],
                collect_idc_ids=[1],)

hook = Simulate(Multi_Agent, env, hook=hook)

[ Info: Normalization is done based on the defined parameter limits.
[ Info: Time simulation run time: 2.0 [s] ~> 20001 steps
[ Info: 1 'classically' controlled source has been initialised.
[ Info: 1 source has been set up in Swing mode.
[ Info: All 'classically' controlled sources have been automatically set up with droop coeficients, and proportional and integral gains.
┌ Warning: The state(s) ["load1_i_L_c"] exceeded limit(s)
│                     -> episode abort
└ @ ElectricGrid C:\Users\marmey\Documents\dare\src\electric_grid_env.jl:516
┌ Warning: Corresponding limit(s): [1.844626385704037],
│                     corresponding index: [15]
└ @ ElectricGrid C:\Users\marmey\Documents\dare\src\electric_grid_env.jl:518


DataHook(false, "episode_data/", [-200.0 -1000.0 … 0.0 0.0; 20000.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … -119.72274732199115 -20000.0; 0.0 0.0 … 0.6181234982183796 0.0], [1000.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0], Any[0.001, 5.0e-5, 0.0001, 5.0e-5, 1.6177996838533155, 0.001, 5.0e-5, 0.0001, 5.0e-5, 1.6177996838533155, 0.001, 5.0e-5, 0.0001, 5.0e-5, 1.6177996838533155], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], Any[], 1077×6 DataFrame
  Row │ episode  time     source1_vdc  source1_idc  reward   done  
      │ Int64    Float64  Float64      Float64      Float64  Bool  
──────┼────────────────────────────────────────────────────────────
    1 │       1   0.0            0.0   0.0              0.0  false
    2 │       1   0.0001         0.0   0.0              0.0  false
    3 │       1   0.0002     10171.2   0.0              0.0  false
    4 │       1   0.0003     10171.2   9.97113e-7       0.0  false
    5 │       1   0.0004     10171.2   1.04033e-5       0.0  false


In [25]:
p = RenderHookResults(hook=hook,
                      vdc_to_plot=[1],
                      idc_to_plot=[1],
                      return_plot=false);

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields height, legend, margin, plot_bgcolor, template, width, xaxis, yaxis, and yaxis2"